#Content Based Recommendation System

##Inisiasi

In [ ]:
pip install html2text

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import scipy
import pandas as pd
import math
import re
import random
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
import sklearn
import html2text
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

##Preprocessing

In [ ]:
datasagara =  pd.read_csv('/content/drive/MyDrive/Datasets/Sagara/wpc0_posts.csv')
datasagara

,ID,post_author,post_date,post_date_gmt,post_content,post_title,post_excerpt,post_status,comment_status,ping_status,post_password,post_name,to_ping,pinged,post_modified,post_modified_gmt,post_content_filtered,post_parent,guid,menu_order,post_type,post_mime_type,comment_count
0,2,1,2020-08-25 05:12:33,2020-08-25 05:12:33,<!-- wp:paragraph -->\n<p>This is an example p...,Sample Page,NaN,publish,closed,open,NaN,sample-page,NaN,NaN,2020-08-25 05:12:33,2020-08-25 05:12:33,NaN,0,https://www.sagaratechnology.com/blog/?page_id=2,0,page,NaN,0
1,3,1,2020-08-25 05:12:33,2020-08-25 05:12:33,<!-- wp:heading --><h2>Who we are</h2><!-- /wp...,Privacy Policy,NaN,draft,closed,open,NaN,privacy-policy,NaN,NaN,2020-08-25 05:12:33,2020-08-25 05:12:33,NaN,0,https://www.sagaratechnology.com/blog/?page_id=3,0,page,NaN,0
2,39,1,2020-08-26 11:20:19,2020-08-26 11:20:19,NaN,working,NaN,inherit,open,closed,NaN,working,NaN,NaN,2020-08-26 11:20:19,2020-08-26 11:20:19,NaN,0,https://www.sagaratechnology.com/blog/wp-conte...,0,attachment,image/jpeg,0
3,40,1,2020-08-26 11:21:23,2020-08-26 11:21:23,NaN,red-lines,NaN,inherit,open,closed,NaN,red-lines,NaN,NaN,2020-08-26 11:21:23,2020-08-26 11:21:23,NaN,0,https://www.sagaratechnology.com/blog/wp-conte...,0,attachment,image/png,0
4,43,1,2020-08-27 09:30:05,2020-08-27 09:30:05,NaN,The Fundamentals of UX and UI Design,NaN,inherit,open,closed,NaN,featured-image,NaN,NaN,2020-08-27 10:01:04,2020-08-27 10:01:04,NaN,0,https://www.sagaratechnology.com/blog/wp-conte...,0,attachment,image/gif,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
803,1068,2,2021-07-09 03:55:10,2021-07-09 03:55:10,<!-- wp:paragraph -->\n<p>Economists from the ...,Cryptocurrency Controls 8 Percent of the Finte...,NaN,inherit,closed,closed,NaN,1062-revision-v1,NaN,NaN,2021-07-09 03:55:10,2021-07-09 03:55:10,NaN,1062,https://www.sagaratechnology.com/blog/?p=1068,0,revision,NaN,0
804,1069,2,2021-07-09 07:34:10,2021-07-09 07:34:10,<!-- wp:paragraph -->\n<p>One of the most deve...,Getting to Know Indonesian Fintech Startups,NaN,draft,open,open,NaN,NaN,NaN,NaN,2021-07-09 07:39:42,2021-07-09 07:39:42,NaN,0,https://www.sagaratechnology.com/blog/?p=1069,0,post,NaN,0
805,1070,2,2021-07-09 07:34:02,2021-07-09 07:34:02,Getting to Know Indonesian Fintech Startups,Getting to Know Indonesian Fintech Startups,NaN,inherit,open,closed,NaN,getting-to-know-indonesian-fintech-startups,NaN,NaN,2021-07-09 07:34:08,2021-07-09 07:34:08,NaN,1069,https://www.sagaratechnology.com/blog/wp-conte...,0,attachment,image/jpeg,0
806,1071,2,2021-07-09 07:34:10,2021-07-09 07:34:10,<!-- wp:paragraph -->\n<p>One of the most deve...,Getting to Know Indonesian Fintech Startups,NaN,inherit,closed,closed,NaN,1069-revision-v1,NaN,NaN,2021-07-09 07:34:10,2021-07-09 07:34:10,NaN,1069,https://www.sagaratechnology.com/blog/?p=1071,0,revision,NaN,0


**Dari semua data artikel yang ada, hanya pilih artikel yang sudah di publish & bukan inherit dari artikel lain**

In [ ]:
datasagara = datasagara[datasagara['post_status']=='publish'].reset_index(drop=True)
datasagara

,ID,post_author,post_date,post_date_gmt,post_content,post_title,post_excerpt,post_status,comment_status,ping_status,post_password,post_name,to_ping,pinged,post_modified,post_modified_gmt,post_content_filtered,post_parent,guid,menu_order,post_type,post_mime_type,comment_count
0,2,1,2020-08-25 05:12:33,2020-08-25 05:12:33,<!-- wp:paragraph -->\n<p>This is an example p...,Sample Page,NaN,publish,closed,open,NaN,sample-page,NaN,NaN,2020-08-25 05:12:33,2020-08-25 05:12:33,NaN,0,https://www.sagaratechnology.com/blog/?page_id=2,0,page,NaN,0
1,48,1,2020-08-28 03:11:04,2020-08-28 03:11:04,<!-- wp:paragraph -->\n<p>Many people have hea...,The Fundamentals of UX and UI Design,NaN,publish,open,open,NaN,the-fundamentals-of-ux-and-ui-design,NaN,NaN,2020-09-02 04:56:56,2020-09-02 04:56:56,NaN,0,https://www.sagaratechnology.com/blog/?p=48,0,post,NaN,0
2,62,1,2020-08-28 07:13:19,2020-08-28 07:13:19,<!-- wp:paragraph -->\n<p>A&nbsp;<strong>user ...,How to Create User Persona for Digital Product?,NaN,publish,open,open,NaN,how-to-create-user-persona-for-digital-product,NaN,NaN,2020-09-02 04:55:05,2020-09-02 04:55:05,NaN,0,https://www.sagaratechnology.com/blog/?p=62,0,post,NaN,0
3,67,1,2020-08-28 08:34:31,2020-08-28 08:34:31,"<p id=""e89a"" class=""hu hv bo hw b hx hy hz ia ...",Creating User Journey for Apps Development,NaN,publish,open,open,NaN,creating-user-journey-for-apps-development,NaN,NaN,2020-09-02 04:54:07,2020-09-02 04:54:07,NaN,0,https://www.sagaratechnology.com/blog/?p=67,0,post,NaN,0
4,76,1,2020-08-28 09:57:39,2020-08-28 09:57:39,"<!-- wp:paragraph -->\n<p>Nowadays, people are...",A Use Case for Proper Mobile Apps Development,NaN,publish,open,open,NaN,a-use-case-for-proper-mobile-apps-development,NaN,NaN,2020-09-02 04:57:57,2020-09-02 04:57:57,NaN,0,https://www.sagaratechnology.com/blog/?p=76,0,post,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,1007,2,2021-07-06 05:14:07,2021-07-06 05:14:07,<!-- wp:paragraph -->\n<p>To access informatio...,Difference between Mobile Apps and Mobile Web,NaN,publish,open,open,NaN,difference-between-mobile-apps-and-mobile-web,NaN,NaN,2021-07-06 05:14:07,2021-07-06 05:14:07,NaN,0,https://www.sagaratechnology.com/blog/?p=1007,0,post,NaN,0
95,1022,2,2021-07-07 07:02:38,2021-07-07 07:02:38,"<!-- wp:paragraph -->\n<p>Currently, the busin...",Software House Services and Structure,NaN,publish,open,open,NaN,software-house-services-and-structure,NaN,NaN,2021-07-07 07:26:17,2021-07-07 07:26:17,NaN,0,https://www.sagaratechnology.com/blog/?p=1022,0,post,NaN,0
96,1037,2,2021-07-07 11:12:05,2021-07-07 11:12:05,"<!-- wp:paragraph -->\n<p><a href=""https://www...",Tips for Choosing a Good Software House,NaN,publish,open,open,NaN,tips-for-choosing-a-good-software-house,NaN,NaN,2021-07-08 02:43:09,2021-07-08 02:43:09,NaN,0,https://www.sagaratechnology.com/blog/?p=1037,0,post,NaN,0
97,1042,2,2021-07-09 01:55:05,2021-07-09 01:55:05,"<!-- wp:paragraph -->\n<p><a href=""https://www...",SAGARA FINTECH ROUNDTABLE: How to Take Advanta...,NaN,publish,open,open,NaN,sagara-fintech-roundtable-how-to-take-advantag...,NaN,NaN,2021-07-09 01:55:05,2021-07-09 01:55:05,NaN,0,https://www.sagaratechnology.com/blog/?p=1042,0,post,NaN,0


In [ ]:
datasagara[datasagara['post_parent']!=0]

,ID,post_author,post_date,post_date_gmt,post_content,post_title,post_excerpt,post_status,comment_status,ping_status,post_password,post_name,to_ping,pinged,post_modified,post_modified_gmt,post_content_filtered,post_parent,guid,menu_order,post_type,post_mime_type,comment_count


**Cek ID apakah ada yang duplikat/tidak**

In [ ]:
datasagara['ID'].value_counts()

928    1
368    1
290    1
163    1
811    1
      ..
918    1
218    1
475    1
92     1
603    1
Name: ID, Length: 99, dtype: int64

**Melihat isi artikel**

In [ ]:
datasagara['post_title']

0                                           Sample Page
1                  The Fundamentals of UX and UI Design
2       How to Create User Persona for Digital Product?
3            Creating User Journey for Apps Development
4         A Use Case for Proper Mobile Apps Development
                            ...                        
94        Difference between Mobile Apps and Mobile Web
95                Software House Services and Structure
96              Tips for Choosing a Good Software House
97    SAGARA FINTECH ROUNDTABLE: How to Take Advanta...
98                  5 Advanced Technology of the Future
Name: post_title, Length: 99, dtype: object

In [ ]:
datasagara['post_content']

0     <!-- wp:paragraph -->\n<p>This is an example p...
1     <!-- wp:paragraph -->\n<p>Many people have hea...
2     <!-- wp:paragraph -->\n<p>A&nbsp;<strong>user ...
3     <p id="e89a" class="hu hv bo hw b hx hy hz ia ...
4     <!-- wp:paragraph -->\n<p>Nowadays, people are...
                            ...                        
94    <!-- wp:paragraph -->\n<p>To access informatio...
95    <!-- wp:paragraph -->\n<p>Currently, the busin...
96    <!-- wp:paragraph -->\n<p><a href="https://www...
97    <!-- wp:paragraph -->\n<p><a href="https://www...
98    <!-- wp:paragraph -->\n<p><a href="https://www...
Name: post_content, Length: 99, dtype: object

**Dari konten artikel yang ada di bawah, terlihat bahwa text masih kotor (tidak rapi)**

In [ ]:
datasagara['post_content'][98]

'<!-- wp:paragraph -->\n<p><a href="https://www.youtube.com/watch?v=MHr61r3-sH4" target="_blank" rel="noreferrer noopener">Technology</a> will continue to grow and grow rapidly. The main goal of technological development is usually to make human life easier. But in its development, many positions that were originally able to be done by humans have been shifted to using equipment from technological innovations.</p>\n<!-- /wp:paragraph -->\n\n<!-- wp:paragraph -->\n<p>Technological development will continue to be rapid along with human needs. Planning may be tried years in advance and creatively innovate.</p>\n<!-- /wp:paragraph -->\n\n<!-- wp:paragraph -->\n<p>Here is some cutting-edge technology that can take over the human position:</p>\n<!-- /wp:paragraph -->\n\n<!-- wp:paragraph -->\n<p><strong>Advanced Nano Technology</strong><br>Currently, nanotechnology has been widely known and used for various innovations. From the world of medicine, informatics, PC to the world of beauty using

**Melihat kerapian text apabila menggunakan library html2text**

In [ ]:
print(html2text.html2text(datasagara['post_content'][98]))

[Technology](https://www.youtube.com/watch?v=MHr61r3-sH4) will continue to
grow and grow rapidly. The main goal of technological development is usually
to make human life easier. But in its development, many positions that were
originally able to be done by humans have been shifted to using equipment from
technological innovations.

Technological development will continue to be rapid along with human needs.
Planning may be tried years in advance and creatively innovate.

Here is some cutting-edge technology that can take over the human position:

**Advanced Nano Technology**  
Currently, nanotechnology has been widely known and used for various
innovations. From the world of medicine, informatics, PC to the world of
beauty using nanotechnology. This technology uses an innovation of very small
dimensions and almost atomic dimensions.

**Laundroid**  
This laundroid can help with household chores that will save energy. With this
advanced technology equipment, washing and ironing clothes 

In [ ]:
print(html2text.html2text(datasagara['post_content'][97]))

[Sagara Fintech Roundtable Day
3](https://www.youtube.com/watch?v=R8Ma5radPjI&t=864s) \- Tuesday, July 6th,
2021- [Sagara Technology ](https://www.sagaratechnology.com/en)held a Talk
Show on Fintech by inviting C-levels who are experts in the fields of
technology development from various existing fintech companies. The main
discussion of the talk show is talking about managing our finances by
investing in digital investment platforms for future profits. Starting from
why investing in digital investment platforms is important, to differences
between investing and saving, ways to start investing, as well as how to
manage our finances. Therefore, this discussion aims to educate the public so
that people are more sensitive to technological advances, so they can take
advantage of the technology fintech development properly and wisely. The
following are reviews from the speakers of the Sagara Fintech Roundtable.

### Rieka Handayani, Head of Public Relation TokoCrypto

![](https://www.sagara

In [ ]:
print(html2text.html2text(datasagara['post_content'][96]))

[Software house](https://www.sagaratechnology.com/en) is a company engaged in
the field of software development. The software house itself was born due to
the demands of increasingly rapid technological developments. This can be a
positive thing for software house service enthusiasts because there are more
and more choices that can be taken officially. However, choosing an option
should not be hasty. In order to make the right choice, consider the
following.

### Check Portfolio

In general, long-lasting businesses are good quality businesses. In addition,
pay attention to the clients being handled. The more prestigious the company
that has been a client, the better the quality is usually.

### Customer Reviews

Compare the number of comments that seem satisfied or disappointed. By looking
at the largest number, an assessment will be obtained about the level of
professionalism of the software house in outline.

### Price and Lead Time

Before deciding to use the services of a software 

In [ ]:
print(html2text.html2text(datasagara['post_content'][95]))

Currently, the business is growing in accordance with the increasingly rapid
technological developments, especially now that many have linked it with
software development that serves to simplify the business processes of various
industries that are run. One of them is the Software House which is very
important for business in today's times.  
  
[Software house](https://www.sagaratechnology.com/en) is a company that
focuses on application development (software development). The application can
be developed either by individual actors or by a group such as a legal entity
company. Even to work in a software house company, of course, you must have
special abilities in the IT (Information Technology) field.

### Software House Company Services:

  1. Web-Based Applications Development  
The software house will provide application development services based on the
website and its design. That way they must have excellent programming language
skills in website creation.

  2. Mobile-Based Ap

In [ ]:
print(html2text.html2text(datasagara['post_content'][94]))

To access information through mobile devices, of course, you often use a web
browser or open an application to find the information you need. Indirectly
you have used mobile apps or mobile web on your device. For that, let's find
out more fully what mobile apps / mobile applications and mobile web are.

### Understanding Mobile Web

[Mobile web](https://www.sagaratechnology.com/en) is a browser-based HTML page
that is accessed using portable devices (smartphones or tablets) via cellular
networks such as 3G, 4G, or Wifi. Mobile Web is designed to display data such
as text, images, and videos from a website into a smaller display, namely
mobile devices.

### Understanding Mobile Apps

While [mobile apps](https://www.youtube.com/watch?v=MHr61r3-sH4) are
applications made for mobile devices where users have to download or download
and install from application stores such as the Google Play Store for Android
devices and the Apple App Store for iOS devices and many more application
stores th

**Melihat apakah ada data yang NaN/null. Jika ada, isi data yang NaN dengan string kosong**

In [ ]:
datasagara['ID'].isnull().values.any()

False

In [ ]:
datasagara['post_content'].isnull().values.any()

False

In [ ]:
datasagara['post_title'].isnull().values.any()

True

In [ ]:
datasagara['post_title'] = datasagara['post_title'].fillna('')
datasagara['post_title'].isnull().values.any()

False

**Terapkan html2text preprocessing ke konten artikel**

In [ ]:
for i in range(len(datasagara['post_content'])):
  datasagara['post_content'][i] = html2text.html2text(datasagara['post_content'][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
datasagara['post_content']

0     This is an example page. It's different from a...
1     Many people have heard the term “ **User Exper...
2     A  **user persona**  is a representation of yo...
3     A  **user journey**  is a timeline of user act...
4     Nowadays, people are spending more time on the...
                            ...                        
94    To access information through mobile devices, ...
95    Currently, the business is growing in accordan...
96    [Software house](https://www.sagaratechnology....
97    [Sagara Fintech Roundtable Day\n3](https://www...
98    [Technology](https://www.youtube.com/watch?v=M...
Name: post_content, Length: 99, dtype: object

In [ ]:
print(datasagara['post_content'][98])

[Technology](https://www.youtube.com/watch?v=MHr61r3-sH4) will continue to
grow and grow rapidly. The main goal of technological development is usually
to make human life easier. But in its development, many positions that were
originally able to be done by humans have been shifted to using equipment from
technological innovations.

Technological development will continue to be rapid along with human needs.
Planning may be tried years in advance and creatively innovate.

Here is some cutting-edge technology that can take over the human position:

**Advanced Nano Technology**  
Currently, nanotechnology has been widely known and used for various
innovations. From the world of medicine, informatics, PC to the world of
beauty using nanotechnology. This technology uses an innovation of very small
dimensions and almost atomic dimensions.

**Laundroid**  
This laundroid can help with household chores that will save energy. With this
advanced technology equipment, washing and ironing clothes 

**Merapikan konten artikel dengan:**


1.   **Mengubah semua karakter menjadi lowercase**
2.   **Mengganti \n dengan spasi**
3.   **Menghilangkan simbol dan angka**
4.   **Menghilangkan url link & gambar**
5.   **Menghilangkan stopwords (kata yang tidak mempunyai informasi seperti kata sambung) bahasa indonesia & inggris**

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^a-z ]')
stopworda = set(stopwords.words('indonesian'))
stopwordb = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = text.replace('\n',' ')
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r'[\S]+\.(net|id|com|org|info|edu|gov|uk|de|ca|jp|fr|au|us|ru|ch|it|nel|se|no|es|mil)[\S]*\s?','',text)
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"\S+.jpeg", "", text)
    text = re.sub(r"\S+.png", "", text)
    text = re.sub(r"\S+.jpg", "", text)
    text = clean_spcl.sub(' ', text)
    text = clean_symbol.sub('', text)
    text = ' '.join(word for word in text.split() if word not in stopworda)
    text = ' '.join(word for word in text.split() if word not in stopwordb)
    return text

datasagara['post_contentCleaned'] = datasagara['post_content'].apply(clean_text)

**Melihat hasil text**

In [ ]:
datasagara['post_contentCleaned'][98]

'technology continue grow grow rapidly main goal technological development usually make human life easier development many positions originally able done humans shifted using equipment technological innovations technological development continue rapid along human needs planning may tried years advance creatively innovate cuttingedge technology take human position advanced nano technology currently nanotechnology widely known used various innovations world medicine informatics pc world beauty using nanotechnology technology uses innovation small dimensions almost atomic dimensions laundroid laundroid help household chores save energy advanced technology equipment washing ironing clothes done instantly easily laundroid robot created able wash clothes produce folded form mind reading equipment equipment used pcs take functions hands fingers generally carry process pc must press certain keys keyboard latest technology dont tired pressing keyboard buttons operating pc use mind hologram gene

**Melakukan proses lemmatization (mengganti verb 2 dan verb 3 menjadi verb 1, mengubah verb ganda menjadi verb tunggal, mengubah adverb, noun, adjective dst) dari setiap kata dengan library nltk**

In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
VERB_CODES = {'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'}
ADVERB_CODES = {'RB', 'RBR', 'RBS'}
NOUN_CODES = {'NN', 'NNS', 'NNP', 'NNPS'}
ADJ_CODES = {'JJ', 'JJS', 'JJR'}

In [ ]:
def preprocess_sentences(text):
  text = text.lower()
  temp_sent =[]
  words = nltk.word_tokenize(text)
  tags = nltk.pos_tag(words)
  for i, word in enumerate(words):
      if tags[i][1] in VERB_CODES: 
          lemmatized = lemmatizer.lemmatize(word, 'v')
      elif tags[i][1] in ADVERB_CODES: 
          lemmatized = lemmatizer.lemmatize(word, 'r')
      elif tags[i][1] in NOUN_CODES: 
          lemmatized = lemmatizer.lemmatize(word, 'n')
      elif tags[i][1] in ADJ_CODES: 
          lemmatized = lemmatizer.lemmatize(word, 'a')
      else:
          lemmatized = lemmatizer.lemmatize(word)
      if lemmatized not in stop_words and lemmatized.isalpha():
          temp_sent.append(lemmatized)
          
  finalsent = ' '.join(temp_sent)
  finalsent = finalsent.replace("n't", " not")
  finalsent = finalsent.replace("'m", " am")
  finalsent = finalsent.replace("'s", " is")
  finalsent = finalsent.replace("'re", " are")
  finalsent = finalsent.replace("'ll", " will")
  finalsent = finalsent.replace("'ve", " have")
  finalsent = finalsent.replace("'d", " would")
  return finalsent
  
datasagara['post_contentLemma'] = datasagara['post_contentCleaned'].apply(preprocess_sentences)

**Melihat hasil lemmatization**

In [ ]:
datasagara['post_contentLemma'][98]

'technology continue grow grow rapidly main goal technological development usually make human life easier development many position originally able human shift use equipment technological innovation technological development continue rapid along human need planning may try year advance creatively innovate cuttingedge technology take human position advance nano technology currently nanotechnology widely know use various innovation world medicine informatics pc world beauty use nanotechnology technology use innovation small dimension almost atomic dimension laundroid laundroid help household chore save energy advance technology equipment wash iron clothes instantly easily laundroid robot create able wash clothes produce fold form mind reading equipment equipment use pc take function hand finger generally carry process pc must press certain key keyboard late technology dont tire press keyboard button operate pc use mind hologram generally hologram technology animation see films time watch

**Stemming (menghilangkan imbuhan) dari setiap kata dengan library nltk**

In [ ]:
porter = PorterStemmer()
def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

In [ ]:
datasagara['post_contentStemmed'] = datasagara['post_contentLemma'].apply(stemSentence)

**Melihat hasil stemming**

In [ ]:
datasagara['post_contentStemmed'][98]

'technolog continu grow grow rapidli main goal technolog develop usual make human life easier develop mani posit origin abl human shift use equip technolog innov technolog develop continu rapid along human need plan may tri year advanc creativ innov cuttingedg technolog take human posit advanc nano technolog current nanotechnolog wide know use variou innov world medicin informat pc world beauti use nanotechnolog technolog use innov small dimens almost atom dimens laundroid laundroid help household chore save energi advanc technolog equip wash iron cloth instantli easili laundroid robot creat abl wash cloth produc fold form mind read equip equip use pc take function hand finger gener carri process pc must press certain key keyboard late technolog dont tire press keyboard button oper pc use mind hologram gener hologram technolog anim see film time watch star war see hologram technolog technolog dimension commun tool seem introduc caller front call face face connect quantum pc pc innov ad

**Set ID sebagai index untuk memudahkan pemrosesan**

In [ ]:
datasagara.set_index('ID', inplace=True)

In [ ]:
indices = pd.Series(datasagara.index)
indices[:50]

0       2
1      48
2      62
3      67
4      76
5      92
6      95
7      98
8     104
9     107
10    108
11    109
12    112
13    141
14    146
15    151
16    154
17    160
18    163
19    166
20    181
21    185
22    188
23    197
24    201
25    204
26    209
27    213
28    218
29    269
30    290
31    296
32    298
33    368
34    371
35    376
36    390
37    412
38    419
39    446
40    451
41    461
42    462
43    467
44    471
45    475
46    489
47    542
48    551
49    561
Name: ID, dtype: int64

##Pembuatan model Recommendation System

**Menghitung TF-IDF dari tiap artikel. TF-IDF menghitung bobot nilai setiap kata yang ada di artikel, semakin sering kata tersebut muncul di artikel semakin tinggi bobot nilainya.**

In [ ]:
stopwords_list = stopwords.words('english')

vectorizer = TfidfVectorizer(analyzer='word',
                     ngram_range=(1, 2),
                     min_df=0.003,
                     max_df=0.5,
                     max_features=5000,
                     stop_words=stopwords_list)

tfidf_matrix = vectorizer.fit_transform(datasagara['post_title'] + "" + datasagara['post_contentStemmed'])
tfidf_feature_names = vectorizer.get_feature_names()
tfidf_matrix

<99x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 23659 stored elements in Compressed Sparse Row format>

**Membuat matriks TF-IDF lalu menghitung cosine similarity. Cosine similarity menghitung kesamaan TF-IDF antara dua artikel. Semakin mirip isi konten artikel, semakin tinggi nilai cosine similaritynya (skala 0-1)**

In [ ]:
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cos_sim

array([[1.        , 0.08307488, 0.02480366, ..., 0.01855651, 0.0212821 ,
        0.00892693],
       [0.08307488, 1.        , 0.12926002, ..., 0.04073382, 0.05092041,
        0.03680044],
       [0.02480366, 0.12926002, 1.        , ..., 0.02007934, 0.03064819,
        0.01333269],
       ...,
       [0.01855651, 0.04073382, 0.02007934, ..., 1.        , 0.0875417 ,
        0.03821381],
       [0.0212821 , 0.05092041, 0.03064819, ..., 0.0875417 , 1.        ,
        0.05039803],
       [0.00892693, 0.03680044, 0.01333269, ..., 0.03821381, 0.05039803,
        1.        ]])

**Membuat sistem rekomendasi dari nilai cosine similarity yang sudah dibuat.**

In [ ]:
def recommendations(id, cos_sim = cos_sim):
    recommended = []
    idx = indices[indices == id].index[0]
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)
    top_10_indexes = list(score_series.iloc[1:11].index)
    for i in top_10_indexes:
        recommended.append(list(datasagara.index)[i])
        
    return list(zip(recommended,score_series.iloc[1:11]))

**Contoh rekomendasi apabila pembaca membaca artikel dengan ID 791**

In [ ]:
recommendations(791)

[(664, 0.4303892602086362),
 (814, 0.2916988208810198),
 (869, 0.20779369236658776),
 (781, 0.19235903209805608),
 (1042, 0.18299029988872859),
 (839, 0.17515276914736755),
 (889, 0.17265180309497788),
 (854, 0.13752057780439678),
 (795, 0.13688889311953278),
 (412, 0.1367559407349779)]

**Dari hasil rekomendasi diatas, tiga artikel yang paling direkomendasikan adalah artikel dengan ID 664, 814, dan 869. Untuk mengecek apakah rekomendasi sudah sesuai, cek isi konten dengan ID 791, 664, 814, dan 869 apakah isinya mirip/tidak.**

In [ ]:
datasagara['post_title'][791]

'Getting to know Cryptocurrency Investments'

In [ ]:
print(datasagara['post_content'][791])

While blockchain as a digital platform continues to be explored,
[cryptocu](http://en.wikipedia.org/wiki/Cryptocurrency#:~:text=A%20cryptocurrency%2C%20crypto-
currency%2C,the%20creation%20of%20additional%20coins%2C)[rrency](https://www.sagaratechnology.com/en)
blockchain continues to flourish among consumers. In addition to dealing with
transactions, ICOs (Initial Coin Offerings) is also a process that is starting
to be found among business people. ICO is a fundraising process to build
blockchain-based applications, in various forms ranging from new
cryptocurrencies, smart contracts, to smart ledgers _._   _ _  

In Indonesia alone, there are at least three businesses that have tried to
conduct ICOs, including Tokenomy, Pundi X, and Vexanium. Each offers different
characteristics, with different business processes.

## **Invest in cryptocurrencies**

In Indonesia, there are already several platforms that serve the process of
making crypto wallets. This includes making it easier for us

In [ ]:
datasagara['post_title'][664]

'Get to Know a Cryptocurrency'

In [ ]:
print(datasagara['post_content'][664])

For those who are new to investing, discussing
[Cryptocurrency](https://en.wikipedia.org/wiki/Cryptocurrency) is a bit of a
headache. Not finished learning about stocks or mutual funds, the world is now
planning to adopt Cryptocurrency for payment instruments. However, you don't
need to worry. As long as you have the desire to learn and are always careful
when making decisions, so your Crypto money investment will be successful.
Well, before starting investing, it helps us know what Cryptocurrency is. The
following is an understanding of Cryptocurrency and its brief history.

**What is Cryptocurrency?**

Reporting from various sources, Cryptocurrency is a digital currency that is
used to transact virtually via the internet network. In addition, it is
explained that Cryptocurrency is decentralized, which means neither party will
be an intermediary in a transaction. Payments are also made directly between
the recipient and the sender (peer to peer). To make transactions easier,
Cryptocur

In [ ]:
datasagara['post_title'][814]

'Advantages and Disadvantages of Cryptocurrency'

In [ ]:
print(datasagara['post_content'][814])

[Cryptocurrency](https://www.sagaratechnology.com/blog/getting-to-know-
cryptocurrency-investment/), a virtual currency as part of blockchain
technology, is becoming a hot topic for everyone in all parts of the world,
including Indonesia.

Cryptocurrency can be used for investment purposes, trading, mining, to
transaction tools. Even so, cryptocurrency has its advantages and
disadvantages.

**The first** advantage of this currency is that it is universal. We can use
this currency without any conditions, including when they want to buy it.

**The second** advantage, this currency allows fast transactions.
Cryptocurrency do not require a third party such as a bank to act as an
intermediary in conventional financial transactions.

**The next** advantage, cryptocurrency is transparent. Users can see all
transactions made. However, only the transaction value can be known.

**Lastly** , cryptocurrency have the advantage that they can be traded 7 days
24 hours. This means that cryptocurrency 

In [ ]:
datasagara['post_title'][869]

'Dogecoin, a Popular Cryptocurrency other than Bitcoin'

In [ ]:
print(datasagara['post_content'][869])

When it comes to cryptocurrencies or digital currencies, most people
immediately think of Bitcoin. But besides Bitcoin, there is also another
cryptocurrency that is no less popular, namely Dogecoin.

According to [CNN](https://edition.cnn.com/), this digital currency has been
circulating quite a lot with a value of almost 50 billion United States (US)
dollars or around Rp. 700 trillion. According to CoinMarketCap, it is the 5th
most valuable cryptocurrency on the market, and its value has increased by
more than 6 thousand percent this year.

The price of Dogecoin doubled again after
[Tesla](https://www.sagaratechnology.com/blog/tesla-buys-bitcoin-
worth-21-trillion/) CEO Elon Musk tweeted about the coin. Demand for Dogecoin
has also surged so much this week that it has caused disruptions to the
Robinhood cryptocurrency trading system.

Here are some things you should to know about Dogecoin.

#### 1\. Dogecoin and other cryptocurrencies

Like all cryptocurrencies, Dogecoin is a digital 

**Dari rekomendasi diatas, terlihat bahwa sistem rekomendasi berhasil merekomendasikan artikel yang mirip. Semua artikel membahas cryptocurrency**